<div style="text-align: center;">
<h1>Reel Realities: How Gender and Age Shape Success Across Box Office and Streaming Platforms</h1>
</div>

### <u>Imports</u>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import ast
import re
import numpy as np
import os
import seaborn as sns
import math
from sklearn.preprocessing import StandardScaler, MinMaxScaler

### 1. <u>Data cleaning and pre-processing</u>

#### 1.1 CMU Dataset

We will use three files of the CMU dataset:
1. "plot_summaries.txt" gives us the plots of the movies.
2. "movie.metadata.tsv" gives us information about the languages, countries, and genres of the movies.
3. "character.metadata.tsv" gives us information about actors and the characters they play.

In [ ]:
path_plot_summaries = os.path.join("data","CMU","plot_summaries.txt")
plot_summaries_df = pd.read_csv(path_plot_summaries, delimiter="\t", names = ["Wikipedia_movie_ID", "Plot Summaries"])

print(f"The plot summaries dataframe has {len(plot_summaries_df):,} values.")

In [ ]:
null_id = plot_summaries_df["Wikipedia_movie_ID"].isnull().sum()
empty_id = plot_summaries_df[plot_summaries_df["Wikipedia_movie_ID"] == ""]["Wikipedia_movie_ID"].sum()
null_plot = plot_summaries_df["Plot Summaries"].isnull().sum()
empty_plot = plot_summaries_df[plot_summaries_df["Plot Summaries"] == ""]["Plot Summaries"].sum()

In [ ]:
print(f"Number of null values in 'Wikipedia_movie_ID': {null_id}")
print(f"Number of empty values in 'Wikipedia_movie_ID': {empty_id}")
print(f"Number of null values in 'Plot Summaries': {null_plot}")
print(f"Number of empty values in 'Plot Summaries': {empty_plot}")

In [ ]:
column_names = ["Wikipedia_movie_ID",
                "Freebase_movie_ID",
                "Movie_name",
                "Movie_release_date",
                "Movie_box_office_revenue",
                "Movie_runtime",
                "Movie_languages",
                "Movie_countries",
                "Movie_genres"]

movie_metadata_df = pd.read_csv("data/CMU/movie.metadata.tsv", delimiter='\t', names = column_names)
print(f"The movie metadata dataframe has {len(movie_metadata_df):,} values.")

In [ ]:
movie_metadata_df["Movie_name"] = (movie_metadata_df["Movie_name"]
                    .str.strip()  #Remove leading/trailing whitespace
                    .str.lower()  #Convert to lowercase
                    .replace("", np.nan)  #Replace empty strings with NaN
                   )

In [ ]:
# Movie languages, countries, and genres are formatted as a dictionary, but they are actually a string.
# We convert them to comma spereated strings. 
movie_metadata_df["Movie_languages"] = movie_metadata_df["Movie_languages"].apply(ast.literal_eval)
movie_metadata_df["Movie_languages"] = movie_metadata_df["Movie_languages"].apply(lambda x: ', '.join(x.values()))

movie_metadata_df["Movie_countries"] = movie_metadata_df["Movie_countries"].apply(ast.literal_eval)
movie_metadata_df["Movie_countries"] = movie_metadata_df["Movie_countries"].apply(lambda x: ', '.join(x.values()))

movie_metadata_df["Movie_genres"] = movie_metadata_df["Movie_genres"].apply(ast.literal_eval)
movie_metadata_df["Movie_genres"] = movie_metadata_df["Movie_genres"].apply(lambda x: ', '.join(x.values()))

In [ ]:
# Cleaning
def clean_string_list(lst):
    # Check if lst is a list
    if isinstance(lst, list):
        return [s.strip().lower() if isinstance(s, str) and s.strip() != "" else np.nan for s in lst]
    elif isinstance(lst, str): 
        return lst.strip().lower()
    else:
        return np.nan 

movie_metadata_df["Movie_languages"] = movie_metadata_df["Movie_languages"].apply(clean_string_list)
movie_metadata_df["Movie_countries"] = movie_metadata_df["Movie_countries"].apply(clean_string_list)
movie_metadata_df["Movie_genres"] = movie_metadata_df["Movie_genres"].apply(clean_string_list)

In [ ]:
#There are inconsistencies in the date format: some rows have the date as year only, others as year-month, and some as year-month-day. We convert them all to the year format only since the month and day are not relevant to our analysis.
full_date_pattern = r'^\d{4}-\d{2}-\d{2}$' #Matches YYYY-MM-DD
year_month_pattern = r'^\d{4}-\d{2}$' #Matches YYYY-MM
year_only_pattern = r'^\d{4}$' #Matches YYYY

def identify_pattern(date):
    if pd.isna(date):
        return "Missing"
    elif re.match(full_date_pattern, date):
        return "Full Date (YYYY-MM-DD)"
    elif re.match(year_month_pattern, date):
        return "Year & Month Date (YYYY-MM)"
    elif re.match(year_only_pattern, date):
        return "Year Only (YYYY)"
    else:
        return "Other"

movie_metadata_df["Pattern"] = movie_metadata_df["Movie_release_date"].apply(identify_pattern)
pattern_summary = movie_metadata_df.groupby("Pattern").size().reset_index(name="Count")

print("Pattern Summary:")
print(pattern_summary)

In [ ]:
movie_metadata_df["Movie_release_date"] = movie_metadata_df["Movie_release_date"].apply(lambda x: str(x)[:4] if pd.notnull(x) else None)
movie_metadata_df = movie_metadata_df.drop(columns=["Pattern"])

In [ ]:
n_null_movie_metadata = ((movie_metadata_df.isnull().sum() / len(movie_metadata_df)) * 100).apply(lambda x: f"{x:,.2f}%")

print(
    f"Percentage of null values per column:\n"
    f"movie_metadata_df:\n{n_null_movie_metadata}"
)

In [ ]:
character_metadata_path = os.path.join("data","CMU","character.metadata.tsv")
character_metadata = pd.read_csv(character_metadata_path, delimiter="\t", header = None)
character_metadata.columns = [
    'Wikipedia_movie_ID', 
    'Freebase_movie_ID', 
    'Movie_release_date', 
    'Character_name', 
    'Actor_date_of_birth', 
    'Actor_gender', 
    'Actor_height', 
    'Actor_ethnicity', 
    'Actor_name', 
    'Actor_age_at_movie_release', 
    'Freebase_character_actor_map_ID', 
    'Freebase_character_ID', 
    'Freebase_actor_ID'
]

In [ ]:
character_metadata['Actor_name']= character_metadata['Actor_name'].fillna('unknown')
character_metadata["Actor_name"] = character_metadata["Actor_name"].str.lower()

# Ensures that the date columns are correctly interpreted as datetime objects in pandas. 
character_metadata['Movie_release_date'] = pd.to_datetime(character_metadata['Movie_release_date'], errors='coerce').dt.date
character_metadata['Actor_date_of_birth'] = pd.to_datetime(character_metadata['Actor_date_of_birth'], errors='coerce').dt.date

# Drop rows where 'Movie_release_date' or 'Actor_date_of_birth' is NaT
character_metadata.dropna(subset=['Movie_release_date', 'Actor_date_of_birth'], inplace=True)

# Calculate Actor_age_at_movie_release if missing and data is available
character_metadata['Actor_age_at_movie_release'] = character_metadata.apply(
    lambda row: (row['Movie_release_date'] - row['Actor_date_of_birth']).days // 365 if pd.isnull(row['Actor_age_at_movie_release']) else row['Actor_age_at_movie_release'],
    axis=1
)

#Drop unnecessary columns , heere the actors' heights
character_metadata.drop(columns=['Actor_height', 'Actor_ethnicity', 'Character_name'], inplace=True)


In [ ]:
print(f"Movies without actor gender data: {character_metadata['Actor_gender'].isnull().sum()}") 

We therefore drop the mvies with no information the gender of the actors.

In [ ]:
character_metadata.dropna(subset=['Actor_gender'], inplace=True)

Now, let's groupe the actors together by creating a *Cast* column on the *movie_metadata_df*. 

In [ ]:
cast_per_movie = character_metadata.groupby('Freebase_movie_ID')['Actor_name'].apply(lambda x: ', '.join(x)).reset_index()
cast_per_movie = cast_per_movie.rename(columns={'Actor_name': 'Cast'})

movie_metadata_df = movie_metadata_df.merge(cast_per_movie, on='Freebase_movie_ID', how='left')

In [ ]:
missing_cast_count = movie_metadata_df["Cast"].isna().sum()
total_movies = len(movie_metadata_df)
print(f"Movies without actor data: {missing_cast_count} out of {total_movies} ({missing_cast_count / total_movies * 100:.2f}%)")

We decide to drop the movies without actor data since it is one of the most important data of ur study. 

In [ ]:
movie_metadata_df = movie_metadata_df.dropna(subset = ['Cast'])

In [ ]:
# Create columns for male, female actor counts
actor_counts = character_metadata.groupby('Freebase_movie_ID')['Actor_gender'].value_counts().unstack(fill_value=0)
actor_counts = actor_counts.rename(columns={'M': 'Male_actors', 'F': 'Female_actors'})

# Ensure all columns are present (set missing columns to 0 if necessary)
for col in ['Male_actors', 'Female_actors']:
    if col not in actor_counts.columns:
        actor_counts[col] = 0

actor_counts = actor_counts.reset_index()

# Merge actor counts with the movie_metadata_df DataFrame
movie_metadata_df = movie_metadata_df.merge(actor_counts, on='Freebase_movie_ID', how='left')

# Fill NaN values with 0 
movie_metadata_df[['Male_actors', 'Female_actors']] = movie_metadata_df[['Male_actors', 'Female_actors']].fillna(0)

In [ ]:
# Calculate the percentage of female actors in each movie and round to two decimal places
movie_metadata_df['Female_actor_percentage'] = (
    (movie_metadata_df['Female_actors'] / 
     (movie_metadata_df['Female_actors'] + movie_metadata_df['Male_actors'])) * 100
).round(2)

#### 1.2 IMDB Dataset

We will use two IMDB datasets to decribe movies:
1. "title.ratings.tsv" gives us the ratings of the movies as voted by viewers. 
2. "title.basics.tsv", indexes into "title.ratings.tsv" using a alphanumeric unique identifier of the title. It gives general information about the movie such as runtime, release date and adult rating.
3. "title.crew.tsv", indexes into the previous two using the same alphanumeric unique identifier of the title. It gives information on the directors and writers of the movie.

Reference:
Internet Movie Database. (2024). IMDb non-commercial datasets. Retrieved from https://developer.imdb.com/non-commercial-datasets/

In [ ]:
# Loading the datasets. Null values are represented using "\N".
IMDB_ratings_path = os.path.join("data","IMDB","title.ratings.tsv")
IMDB_ratings_df = pd.read_csv(IMDB_ratings_path, delimiter="\t", na_values="\\N", names=["tconst", "Average_ratings", "Num_votes"], low_memory= False)

IMDB_titles_path = os.path.join("data","IMDB","title.basics.tsv")
column_names_basics = ["tconst",
                       "Title_type",
                       "Primary_title",
                       "Original_title",
                       "Is_adult",
                       "Start_year",
                       "End_year",
                       "Movie_runtime",
                       "Movie_genres"]
IMDB_basics_df = pd.read_csv(IMDB_titles_path, delimiter="\t", na_values="\\N", low_memory=False, names=column_names_basics)

IMDB_crew_path = os.path.join("data","IMDB","title.crew.tsv")
IMDB_crew_df = pd.read_csv(IMDB_crew_path, delimiter="\t", na_values="\\N", low_memory=False, names=["tconst", "Directors", "Writers"])

In [ ]:
# Checking the lengths of the datasets
print(f"Length of IMDB_ratings_df: {len(IMDB_ratings_df):,}\n"
      f"Length of IMDB_basics_df: {len(IMDB_basics_df):,}\n"
      f"Length of IMDB_crew_df: {len(IMDB_crew_df):,}")

Before dealing with the null values we will merge the dataframes together using the alphanumeric unique identifier.

In [ ]:
# Merging all three datasets.
IMDB_merged_df = pd.merge(IMDB_ratings_df, IMDB_basics_df, how="inner", left_on="tconst", right_on="tconst")
IMDB_merged_df = pd.merge(IMDB_merged_df, IMDB_crew_df, how="inner", on="tconst")

print(f"The resulting merged dataframe has length: {len(IMDB_merged_df):,}.")
print(f"{len(IMDB_ratings_df)-len(IMDB_merged_df):,} rows were lost in the merging process.")

We can see we do not lose a lot of rows with respect to the IMDB_ratings_df dataframe.

Next, we look at titleType. These dataframes do not only have movies but also short movies, tv shows, episodes. The next step is thus to filter only movies.

In [ ]:
# Filtering movies from the list of titles.
IMDB_merged_df = IMDB_merged_df[IMDB_merged_df["Title_type"] == "movie"]

print(f"There are {len(IMDB_merged_df):,} movies in the resulting dataframe.")

Formatting the strings appropriately.

In [ ]:
IMDB_merged_df["Original_title"] = IMDB_merged_df["Original_title"].apply(clean_string_list)
IMDB_merged_df["Primary_title"] = IMDB_merged_df["Primary_title"].apply(clean_string_list)

We can now look at null values in the merged IMDB dataframe.

In [ ]:
# Checking the percentage of null values in the dataset.
n_null_IMDB = ((IMDB_merged_df.isnull().sum() / len(IMDB_merged_df)) * 100).apply(lambda x: f"{x:,.2f}%")

print(
    f"Percentage of null values per column:\n"
    f"IMDB_ratings_df:\n{n_null_IMDB}"
)

The end year is always missing. Other than that the proportion of missing values is very small (< 12%). End year does not have any useful information for our intended analysis and can thus be dropped. We can also drop the titleType column since we know they are all movies after the filtering that was done above.

In [ ]:
# Dropping unnecessary columns.
IMDB_merged_df = IMDB_merged_df.drop(columns=["End_year", "Title_type"], axis=1)

print(f"The resulting dataframe has {len(IMDB_merged_df):,} rows.")

We will use ratings from the IMDB dataset during our study. However, these ratings are based on viewer votes. Initially we thought of discarding rows with too few votes. However, there could be a link between number of votes and number of views of a movie (although definitely not a direct one). We thus decided to keep all rows for the analysis.

#### 1.3 Merging the datasets

##### 1.3.1 Merging IMDB and CMU Movies

In [ ]:
# Merging on the original title.
merge1 = pd.merge(IMDB_merged_df, movie_metadata_df, how="inner", left_on="Original_title", right_on="Movie_name")
# Merging on the primary title.
merge2 = pd.merge(IMDB_merged_df, movie_metadata_df, how="inner", left_on="Primary_title", right_on="Movie_name")

# Concatenating and dropping duplicates that appear from movies with the same Original_title and Primary_title.
movie_df = pd.concat([merge1, merge2]).drop_duplicates(subset=["tconst"]).reset_index(drop=True)

print(f"The resulting dataframe has {len(movie_df):,} rows.")

Some columns appear twice. Let's take a look at the proportion of null values in each duplicate column.

In [ ]:
n_null_movie = (movie_df.isnull().sum()/len(movie_df)).apply(lambda x:f"{x:.2%}")

print(f"Percentage of null values per column:\n{n_null_movie}")

We can see:
- Movie_runtime_x and Movie_runtime_y have 6.02% and 11.58% missing values respectively. We will combine the non null values from both these columns into a new column called runtimeMinutes and then drop the previous two columns. 
- Movie_genres_x has 1.54% missing values against 0.00% missing values for Movie_genres_y. Furthermore Movie_genres_y is from the CMU dataset and seems more complete. We will thus drop the genres_x column.

In [ ]:
# Combining all non-null values from Movie_runtime_x and Movie_runtime_y into runtimeMinutes.
movie_df["Movie_runtime"] = movie_df["Movie_runtime_x"].combine_first(movie_df["Movie_runtime_y"])

# Dropping the unnecessary columns.
movie_df.drop(columns=["Movie_runtime_x", "Movie_runtime_y", "Movie_genres_x"], inplace=True)

Let's now see if Movie_name, originalTitle and primaryTitle are all necessary or if there are any redundancies.

In [ ]:
test1 = movie_df["Movie_name"] == movie_df["Primary_title"]  
test2 = movie_df["Movie_name"] == movie_df["Original_title"]

# Checking if there are any movies for which Movie_name is not either in Primary_title or Original_title
print(f"There are {(~(test1 | test2)).sum().item()} movies for which Movie_name is in neither Primary_title ot Original_title.")

We can see the Movie_name column is redundant as its information is either in primaryTitle or in originalTitle. We can thus drop this column.

In [ ]:
movie_df.drop(columns="Movie_name", inplace=True)

We can now also drop movie identifier columns (as everything is already indexed): tconst and Freebase_movie_ID

In [ ]:
movie_df.drop(columns=["tconst", "Freebase_movie_ID"], inplace=True)

To connect this with the plot_summaries_df, we perform a left merge.

In [ ]:
movie_df = pd.merge(movie_df, plot_summaries_df, on="Wikipedia_movie_ID", how="left")

Finally, we can drop Wikipedia_movie_ID, which is also a movie identifier column.

In [ ]:
movie_df.drop(columns="Wikipedia_movie_ID", inplace=True)

##### 1.3.3 Merging Kaggle and The numbers datasets to the rest

We have a problem, a lot of missing values(NaN or 0) in Movie box office revenue in movie_df and no production budget included. 
These two new datasets include information about movies revenue and budget and other attributes which are crucial for our analysis.

- `Kaggle_df`: Contains important details about movies, including budgets, genres, and revenue.
- `TheNumbers_df`: Provides detailed financial data, focusing on budget and box office gross both domestically and worldwide.

In [ ]:
Kaggle_path = os.path.join("data","KaggleSingh","movie_dataset.csv")
Kaggle_df = pd.read_csv(Kaggle_path, na_values="\\N", low_memory=False)

TheNumbers_path = os.path.join("data","TheNumbers","tn.movie_budgets.csv")
TheNumbers_df = pd.read_csv(TheNumbers_path, na_values="\\N", low_memory=False)

Quickly view the structure of the dataframes to understand the data types and confirm data has loaded correctly

In [ ]:
Kaggle_df.head()

In [ ]:
TheNumbers_df.head()

We now reduce the complexity of the Kaggle dataset by retaining only the columns necessary for our analysis. Additionally, we clean and convert financials of both datasets to float for consistency with the movie_df. 


In [ ]:
# Dropping the unnecessary columns.
Kaggle_df = Kaggle_df[["budget", "genres", "original_title", "popularity", "production_companies", "production_countries", "revenue"]]
Kaggle_df["budget"] = Kaggle_df["budget"].astype(float)
Kaggle_df["revenue"] = Kaggle_df["revenue"].astype(float)


In [ ]:
Kaggle_df.head()

In [ ]:
# Method to remove '$' and ',' from the financials
def clean_currency_column(column):
    return column.str.replace('[^\d.]', '', regex=True).astype(float)

TheNumbers_df["production_budget"] = clean_currency_column(TheNumbers_df["production_budget"])
TheNumbers_df["domestic_gross"] = clean_currency_column(TheNumbers_df["domestic_gross"])
TheNumbers_df["worldwide_gross"] = clean_currency_column(TheNumbers_df["worldwide_gross"])

In [ ]:
TheNumbers_df.head()

We now perform an outer merge to include all records from both datasets, aligning them based on movie titles.


In [ ]:
# Normalize the title columns for case insensitive matching
Kaggle_df["original_title"] = Kaggle_df["original_title"].str.strip().str.lower()
TheNumbers_df["movie"] = TheNumbers_df["movie"].str.strip().str.lower()

# Merge both dataframes on the title columns
merged_df = pd.merge(Kaggle_df, TheNumbers_df, left_on="original_title", right_on="movie", how='outer')
merged_df.head()


##### Post-merge cleaning

We create the main financial columns ('movie_budget' and 'movie_revenue') by selecting non-null and non-zero values, prioritizing data from TheNumbers dataset when available.


In [ ]:
merged_df['movie_budget'] = np.where((merged_df['production_budget'].notna() & (merged_df['production_budget'] != 0)),
                                     merged_df['production_budget'],
                                     merged_df['budget'])

merged_df['movie_revenue'] = np.where((merged_df['worldwide_gross'].notna() & (merged_df['worldwide_gross'] != 0)),
                                      merged_df['worldwide_gross'],
                                      merged_df['revenue'])

# Drop the original columns that were merged into new ones
merged_df.drop(columns=['budget', 'revenue', 'production_budget', 'worldwide_gross', 'movie'], inplace=True)

merged_df.head()


Filter out movies where both budget and revenue data are missing or zero, as they do not provide value for financial analysis.


In [ ]:
# Create a mask where both columns are either zero or NaN
mask = (
    (merged_df["movie_budget"].isna() | (merged_df["movie_budget"] == 0)) &
    (merged_df["movie_revenue"].isna() | (merged_df["movie_revenue"] == 0))
)

# Apply the mask and drop those rows
df_filtered = merged_df[~mask]

In [ ]:
print("The number of available and relevant movie budget information is " + str(df_filtered["movie_budget"].size))
print("The number of available and relevant movie revenue information is " + str(df_filtered["movie_revenue"].size))

df_filtered.head()

Two merging strategies are employed here to maximize the potential for matching records across the datasets:

Original Title Merge: Merging based on the Original_title from movie_df to the original_title from df_filtered to catch the most direct title matches. 

Secondary Title Merge: Using Primary_title as an alternative matching criterion, knowing that some movies might be listed under alternate titles.


In [ ]:
# Normalize the title columns for case-insensitive matching
movie_df["Original_title"] = movie_df["Original_title"].str.strip().str.lower()
movie_df["Primary_title"] = movie_df["Primary_title"].str.strip().str.lower()

# Merging on the original title.
merge1 = pd.merge(movie_df, df_filtered, how="left", left_on="Original_title", right_on="original_title")
# Merging on the secondary title.
merge2 = pd.merge(movie_df, df_filtered, how="left", left_on="Primary_title", right_on="original_title")

# Concatenating and dropping duplicates that appear from movies with the same Original_title and Primary_title.
final_df = pd.concat([merge1, merge2]).drop_duplicates().reset_index(drop=True)

# Filter out movies where revenue data is missing or zero and prioritize initial revenue
final_df["final_movie_revenue"] = np.where(
    (final_df["Movie_box_office_revenue"].notna() & (final_df["Movie_box_office_revenue"] != 0)),
    final_df["Movie_box_office_revenue"],
    final_df["movie_revenue"]
)

final_df.drop(columns=["Movie_box_office_revenue", "movie_revenue", "original_title", "genres", "id", "release_date"], inplace=True, errors='ignore')

# Renaming the columns to follow the convention
rename_columns = {"popularity":"Popularity", 
                  "production_companies":"Production_companies", 
                  "prodcution_countries":"Production_countries",
                  "domestic_gross":"Domestic_gross",
                  "movie_budget":"Movie_budget",
                  "final_movie_revenue":"Final_movie_revenue",
                  "Movie_genres_y":"Movie_genres"}

final_df.rename(columns=rename_columns, inplace=True)


print(f"The resulting dataframe has {len(final_df):,} rows.")
final_df.head()

### 2. <u>Our success metric</u>

#### Financial Analysis

##### Calculating Return on Investment (ROI)

To provide insights into the financial success of the movies, we calculate the Return on Investment (ROI). This metric is derived by comparing the movie's final revenue to its budget.

- **Condition**: Ensure that both `movie_budget` and `final_movie_revenue` are non-zero and non-null to avoid division errors and ensure data integrity


In [ ]:
condition = (final_df['Movie_budget'].notna() & final_df['Movie_budget'].ne(0) &
             final_df['Final_movie_revenue'].notna() & final_df['Final_movie_revenue'].ne(0))

# Apply the condition to filter the dataframe
final_df = final_df[condition]
print("There is now " + str(final_df["Final_movie_revenue"].size) + " movies to work with in the financial success analysis")

# Calculating ROI column based on profits and budget
final_df["ROI"] = ((final_df["Final_movie_revenue"] - final_df["Movie_budget"]) / final_df["Movie_budget"])

#### Data Normalization and Scaling

Given the varied scales of ratings and ROI, we standardize these features to a common scale using both Standard and Min-Max scalers. 

In [ ]:
# Initialize scalers
standard_scaler = StandardScaler()
minmax_scaler = MinMaxScaler()

final_df["Average_ratings"] = final_df["Average_ratings"].astype(float)
final_df["Normalized_Rating"] = final_df["Average_ratings"] / 10

# Standardize the selected data
final_df["ROI"] = standard_scaler.fit_transform(final_df[["ROI"]])

# Apply Min-Max scaling to the already standardized columns
final_df["ROI"] = minmax_scaler.fit_transform(final_df[["ROI"]])


#### Weighted Success Metric

To evaluate overall movie success, we calculate a weighted sum of normalized ratings and ROI, assigning double the weight to ROI given its financial significance in assessing success.

In [ ]:
# ROI feature will be double the importance of ratings
weights = {'ROI': 0.67, 'Normalized_Rating': 0.33}

# Calculate weighted sum
final_df['movie_success'] = final_df['ROI'] * weights['ROI'] + final_df['Normalized_Rating'] * weights['Normalized_Rating']

final_df.head()

### 3. <u>Gender and age vs success</u>

Now that we have have established our success metric, we can evaluate the effect of actors' gender on the success of a movie. 

The best way we chose in order to conduct our analysis is a **Regression analysis**. Indeed, this will allow us to quantify and understand the relationship between gender, age, and other factors on ROI, ratings and our success metric while controlling for potential confounders.

Before diving into regression, we should perform an **Exploratory Data Analysis (EDA)** to understand the relationships among variables and detect any anomalies.

#### 1.1 Exploratory Data Analysis (EDA)

First, let's list the variable of interest:

Categorical variable:
- *Actor_gender*: An actor is either Male or Female. 
- *isAdult*: A boolean. Returns 1 if it a +18 rated movie
- *Movie_genre*
- *Movie_countries*, 
- *Movie_languages*

Continous variable:
- *Average_female_actor_age*: Represents the average age of the female actresses. 
- *Average_male_actor_age*: Represents the average age of the male actresses.
- *Average_ratings* and *Movie_box_office_revenue*: These are the dependent variables of our studies.    



##### A quick comment about the dependent variables:
The goal of our study is to look at ratings, ROI and our succes metric. However, due to complications with finding enough data for the ROI and success metric estimation we will need to postpone the in-depth analysis to P3.

##### 1.1.1 Univariate Analysis of continuous variables
a. <u>Numerical Analysis</u>: 

As a numerical analysis, we can show a table with tne minimum, maximum, mean, standar deviation (SD) and median of each variable.


In [ ]:
final_df_copy = final_df.copy()

In [ ]:
final_df_copy = final_df_copy[final_df_copy["Movie_languages"] != ""]

In [ ]:
numeric_columns = ["Average_ratings",
                   "Num_votes",
                   "Movie_release_date",
                   "Final_movie_revenue",
                   "Movie_runtime",
                   "Female_actors",
                   "Male_actors"]

In [ ]:
pd.options.display.float_format = "{:,.2f}".format

for col in numeric_columns:
    final_df_copy[col] = pd.to_numeric(final_df_copy[col], errors="coerce")

final_df_copy.dropna(subset=numeric_columns, inplace=True)

# Summary statistics
summary_table = final_df_copy[numeric_columns].agg(["min", "max", "mean", "std", "median"]).T
summary_table.columns = ["Min", "Max", "Mean", "SD", "Median"]
summary_table = summary_table.round(2)

summary_table

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(10, 6))
fig.suptitle("Figure 1: Histogram of Dependent Variables", fontsize=14)
axes = axes.flatten()

dep_var = ["Average_ratings", "Final_movie_revenue", "ROI"]

# Plot histograms for each dependent variables
for i, col in enumerate(dep_var):
    if i < len(axes):  # Ensure we do not exceed the number of axes
        ax = axes[i]
        sns.histplot(final_df_copy, x=col, kde=True, stat="density", ax=ax, bins=15)
        ax.set_title(f"Histogram of {col}")
        ax.set_xlabel(col)
        ax.set_ylabel("Frequency")

# Hide any extra subplots if there are more axes than columns
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

# Make space for the title
plt.tight_layout(rect=[0, 0, 1, 0.95]) 
plt.show()

For P2, we will ignore the ROI metric and look at Average_ratings and Final_movie_revenue.

As we can see in Figure 1, the distribution of *Movie_box_office_revenue* is highly skewed. In order to adresse that, we have to apply a log transformation to it.

In [ ]:
plt.figure(figsize=(6, 4))
sns.histplot(final_df_copy['Final_movie_revenue'].apply(math.log1p), kde=True, edgecolor="black", bins=15)
plt.title("Histogram of log-transformed Final_movie_revenue")
plt.xlabel("log_Movie_box_office_revenue")
plt.ylabel("Frequency")
plt.show()

Now we have successfully obtained a normal distribution for the *Movie_box_office_revenue*. Now, let's plot the independent variables. 

Let's now look at our independent variables.

In [ ]:
# Same as for the dependent variables
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(14, 8))
fig.suptitle("Figure 2: Histogram of Independent Variables", fontsize=14)

axes = axes.flatten()

indep_var = ["Num_votes", "Movie_release_date", "Movie_runtime", "Female_actors", "Male_actors"]

# Plot histograms for each independent variable
for i, col in enumerate(indep_var):
    sns.histplot(final_df_copy, x=col, kde=True, stat="density", ax=axes[i], bins=15)
    axes[i].set_title(f"Histogram of {col}")
    axes[i].set_xlabel(col)
    axes[i].set_ylabel("Frequency")

for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()

In [ ]:
# Apply log transformation to skewed independent variables
final_df_copy['log_Num_votes'] = final_df_copy['Num_votes'].apply(math.log1p)
final_df_copy['log_Movie_runtime'] = final_df_copy['Movie_runtime'].apply(math.log1p)
final_df_copy['log_Female_actors'] = final_df_copy['Female_actors'].apply(math.log1p)
final_df_copy['log_Male_actors'] = final_df_copy['Male_actors'].apply(math.log1p)

# Plot histograms of the transformed variables
fig, axes = plt.subplots(nrows=1, ncols=4, figsize=(18, 6))
fig.suptitle("Figure: Histograms of Log-Transformed Independent Variables", fontsize=14)

sns.histplot(final_df_copy['log_Num_votes'], kde=True, ax=axes[0], color="skyblue", bins=15)
axes[0].set_title("Log of Num_votes")

sns.histplot(final_df_copy['log_Movie_runtime'], kde=True, ax=axes[1], color="skyblue", bins=15)
axes[1].set_title("Log of Movie_runtime")

sns.histplot(final_df_copy['log_Female_actors'], kde=True, ax=axes[2], color="skyblue", bins=15)
axes[2].set_title("Log of Female_actors")

sns.histplot(final_df_copy['log_Male_actors'], kde=True, ax=axes[3], color="skyblue", bins=15)
axes[3].set_title("Log of Male_actors")

plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()


In [ ]:
plt.figure(figsize=(14, 8))
for i, col in enumerate(numeric_columns, 1):
    plt.subplot(4, 2, i)
    sns.boxplot(data=final_df_copy, x=col)
    plt.title(f"Boxplot of {col}")
    
plt.suptitle("Figure 3: Boxplot of Variables")     
plt.tight_layout()
plt.show()

We now have to focus on outliers and removing them for a more accurate modeling. From the boxplots, we chose the remove the outliers of the *Movie_box_office_revenue* and the *Minutes_runtime* variables. To do that, we use the **InterQuartile Range (IQR) method**, which identifies values that are significantly higher or lower than the majority of the data.

The *remove_outliers* function will calculate the IQR for each column specified and removes any rows with values outside the range:

$[Q1 - 1.5 \times \text{IQR}, Q3 + 1.5 \times \text{IQR}]$


In [ ]:
# Define a function to remove outliers based on IQR
def remove_outliers(df, columns):
    for col in columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        # Define lower and upper bounds for outliers
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        # Remove rows with values outside these bounds
        df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]
    return df

# Create a copy of the dataframe without outliers
final_df_no_outliers = remove_outliers(final_df_copy, numeric_columns)

# Display the boxplot again to verify the outliers are removed
plt.figure(figsize=(12, 8))
for i, col in enumerate(numeric_columns, 1):
    plt.subplot(4, 2, i)
    sns.boxplot(data=final_df_no_outliers, x=col)
    plt.title(f"Boxplot of {col} (Without Outliers)")
plt.suptitle("Figure 4: Boxplot of Variables without Outliers")
plt.tight_layout()
plt.show()

In [ ]:
print(f"After removing the outliers and dropping the null values, we are left with {len(final_df_no_outliers):,} movies.")

##### 1.1.2 Bivariate Analysis of continuous variables
a. <u>Correlation matrix</u>:

In [ ]:
correlation_matrix = final_df_no_outliers[numeric_columns].corr()

# Calculate and visualize the correlation matrix
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title("Figure 5: Correlation Matrix of Numeric Variables")
plt.show()

The correlation matrix for the continuous variables in our dataset shows the following relationships:

•⁠  ⁠Female_actors and Male_actors : The correlation coefficient is *0.19*, indicating a weak positive correlation. This suggests that movies with a higher number of female actors also tend to have a higher number of male actors. We note that this could indicate a general tendency for larger casts to include more actors of both genders.

•⁠  ⁠Final_movie_revenue and Male_actors : The correlation coefficient is *0.26*, indicating a moderate positive correlation. This suggests that movies with more male actors might achieve higher box office earnings. 

•⁠  ⁠Final_movie_revenue and Female_actors : The correlation coefficient is *0.16*, indicating a weak positive correlation. This suggests that movies with more female actors also have a slight tendency to generate higher revenue. We note however that this relationship is weaker than with male actors. 

•⁠  ⁠Female_actors and Movie_release_date : The correlation coefficient is *0.15*, indicating a weak positive correlation. This suggests that more recent movies tend to include a higher number of female actors. 

While gender composition affects the correlation, these findings highlight the need for deeper analysis (e.g., regression) to control for potential confounders and isolate the impact of gender on box office success.

b. <u>Scatter plots </u>:

In [ ]:
sns.pairplot(final_df_no_outliers[numeric_columns])

#### The categorical variables
Regression analysis with many categorical variables can quickly get complicated. Indeed using a boolean value for each category except one can cause us to have too many regressors. For variables like isAdult and Gender it is easy as there are only two possibilities. However, for Movie_genre, Movie_countries and Movie_languages we can quickly get into the hundreds. Here is how we plan on managing this:

#### Movie_genre

We set a cutoff threshold of 1000. This means all genres with less than 1000 occurences will be discarded. 

In [ ]:
N = 1000  # Threshold for filtering genres

genres_series = (
    final_df_copy["Movie_genres"]
    .str.replace(r"[\[\]']", "", regex=True)  # Clean the string further for this application
    .str.split(', ')  # Split the genre strings into lists for easier manipulation, using ', ' as the delimiter
    .explode()  # Expand the lists into separate rows with one genre per row
    .str.strip()  # Strip leading and trailing whitespace from each genre
    .str.lower()
)

# Get the count of each genre
genre_counts = genres_series.value_counts()

# Filter out genres with counts < N
filtered_genre_counts = genre_counts[genre_counts >= N]  

# Count the number of genres appearing at least N times
len(filtered_genre_counts) 

Let us check if this restricts our study to too few movies.

In [ ]:
filtered_genre_counts

In [ ]:
# List of genres that meet the threshold
popular_genres = filtered_genre_counts.index.tolist()

# Remove all special characters from the strings to allow proper regex matching
no_special_char = map(re.escape, popular_genres)

# Create regex pattern using the "OR" operator
pattern_genre = '|'.join(no_special_char) 

# Check thanks to the regex pattern if any of the popular genres are in each column of th edataframe
rows_with_popular_genres = final_df_copy["Movie_genres"].str.contains(pattern_genre, case=False, na=False)

print(f"There are {rows_with_popular_genres.sum()} movies that belong to at least one of the {len(filtered_genre_counts)} most popular genres.",
      f"\nThere were {len(final_df_copy)} movies in the dataset before this operation,", 
      f"meaning we lost {(len(final_df_copy) - rows_with_popular_genres.sum())/len(final_df_copy):.2%} of the movies.")

We can drop the movies that have none of the most popular genres. We can also assume that the genres are ordered by relevance. We will thus keep for each movie, the first popular genre that appears.

In [ ]:
def get_main_genre(genres, popular_genres):
    for genre in map(str.strip, genres.split(",")): # Split genres according to "," then remove spaces
        if genre.lower() in popular_genres: # We are going through the list of genres in the order it is presented
            return genre # Return the first genre that is in popular_genres
    
    return np.nan

In [ ]:
final_df_copy = final_df_copy[rows_with_popular_genres]

# Ensuring the popular_genres are lowercase
popular_genres = set(map(str.lower, popular_genres))

# Apply the function to the dataset and store the results in the Main_genre column
final_df_copy["Main_genre"] = final_df_copy["Movie_genres"].apply(lambda x : get_main_genre(x, popular_genres)).str.replace(" ","_")

# Plotting the distribution of main genres
genre_counts = final_df_copy["Main_genre"].value_counts().sort_values(ascending=False)
plt.figure(figsize=(10,6))
plt.bar(genre_counts.index, genre_counts.values, edgecolor='black')
plt.title("Distribution of Main Genres", fontsize=16)
plt.xlabel("Main Genres", fontsize=14)
plt.ylabel("Number of Movies", fontsize=14)
plt.xticks(rotation=45, fontsize=12)
plt.grid(axis="y", alpha=0.5)
plt.tight_layout()
plt.show()

In [ ]:
final_df_copy = pd.get_dummies(final_df_copy, columns=["Main_genre"], drop_first=True) # Reference category is action
final_df_copy.sample(5)

#### Movie_languages

In [ ]:
language_series = (
    final_df_copy['Movie_languages']
    .str.replace(r"[\[\]']", "", regex=True) 
    .str.split(', ')
    .explode()
    .str.strip()
    .str.lower()
    .str.replace(" language", "")
)


language_count = language_series.value_counts()
language_count.head(10).plot(kind='bar', figsize=(10, 6))
plt.xlabel('Languages')
plt.ylabel('Occurrences')
plt.title('Top 10 Languages by Occurrences')
plt.xticks(rotation=0)
plt.yscale('log')
plt.tight_layout()
plt.show()

For Movie_languages we decided to take the 4 most common languages. This allows us to analyze the impact of language. However, we will still have an unbalanced dataset with English being much more prominent than other languages.

We will conduct a first analysis using only English, then we will conduct a second analysis with the top 4 languages after rebalancing the dataset.

In [ ]:
top_languages = language_count.index.tolist()

# Dropping all the rows that do not have one of the top 4 languages.
final_df_copy = final_df_copy[final_df_copy['Movie_languages'].str.contains('|'.join(top_languages), case=False, na=False)]

print(f"We are left with {len(final_df_copy)} datapoints.")

We again have certain rows with multiple languages. We must proceed in a similar way to the genres above to obtain the dummy variables.

Categorizing movies into two groups: 
- those available exclusively in English, 
- those available in other languages (which may include English alongside another language).

In [ ]:
# Giving corresponding labels to each row 'English Only' or 'Not Only English'
final_df_copy['Language_Category'] = final_df_copy['Movie_languages'].apply(
    lambda x: 'English_only' if x.lower() == 'english language' else 'Not_only_english'
)

# Creating the dummy variable and dropping the English_only category to be used as the reference category
final_df_copy = pd.get_dummies(final_df_copy, columns=["Language_Category"], drop_first=True)

# Renaming the column.
final_df_copy.rename(columns={'Language_Category_Not_only_english': 'Is_not_only_english'}, inplace=True)

# Counting the number of movies available in other languages
not_only_eng_count = final_df_copy["Is_not_only_english"].sum()

print(f"There are {not_only_eng_count} movies that are available in other languages and {len(final_df_copy)-not_only_eng_count} movies that are only available in English.")

## OLD METHOD (KEPT FOR REFERENCE)

We can see how unbalanced the dataset is, with 5840 english movies against around 400 movies for the other languages.

#### 1.2 Identifying Potential Confounders
In order to not have bias in our study, it is essential we add the confounders. This means that we have to identify the variables that have an impact on both treatment and outcome, here being the actor gender and the final box office revenue respectivly. The confounders are: 
-  ⁠Movie_genre
-  ⁠Movie_countries
- ⁠Movie_languages

In [ ]:
aaa

### 4. <u>How does it compare to streaming platforms? Are movies made for these platforms different?</u>

In this section, we want to expand our study by analyzing data from various streaming platforms. We focus on the movies of Netflix, Amazon Prime, Disney+, and Hulu. To achieve this, we created a dataset by merging data from multiple sources found on Kaggle.


This dataset structure enables comprehensive analysis across various dimensions such as gender repartition in the cast, the ratings and description. Using these columns, we can investigate how streaming platform movies differ in content and ratings compared to traditional box office movies.



In [ ]:
from src.scripts.streaming import get_streaming_dataframe

streaming_df = get_streaming_dataframe()

In [ ]:
streaming_df.drop_duplicates(inplace=True)

a. <u>Numerical Analysis</u>: 

In [ ]:
numeric_columns_streaming = ["Average_Rating", "Num_Votes", "Movie_release_date", "Movie_runtime", "Is_Adult", "Female_actors", "Male_actors"]
streaming_df_copy = streaming_df.copy()
streaming_df_copy.head()
streaming_df_copy = streaming_df_copy[streaming_df_copy['Movie_runtime'] != 0.0]

In [ ]:

#streaming_df_copy['duration'] = streaming_df_copy['duration'].str.replace(' min', '', regex=False)

streaming_df_copy['Movie_runtime'] = pd.to_numeric(streaming_df_copy['Movie_runtime'], errors="coerce")
summary_table = streaming_df_copy[numeric_columns_streaming].dropna().agg(["min", "max", "mean", "std", "median"]).T
summary_table.columns = ["Min", "Max", "Mean", "SD", "Median"]
summary_table = summary_table.round(2)
summary_table


b. <u>Graphical Analysis</u>: 

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))

sns.histplot(streaming_df_copy, x="Average_Rating", kde=True, stat="density", ax=ax, bins=15)
ax.set_title("Histogram of averageRating")
ax.set_xlabel("Average_Rating")
ax.set_ylabel("Frequency")

plt.tight_layout(rect=[0, 0, 1, 0.95]) 
plt.show()

The streaming ratings have a more balanced distribution than the movie dataframe. This contrast highlights different success metrics: streaming platforms rely on user ratings for a measure of success and box office movies are judged on revenue.
It's more relevant to study the success of a streaming movie based on ratings, we dont need necessarly to have the revenue.

In [ ]:

fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(14, 8))
fig.suptitle("Figure 2: Histogram of Independent Variables", fontsize=14)

axes = axes.flatten()

indep_var = ["Num_Votes", "Movie_release_date", "Movie_runtime", "Is_Adult", "Female_actors", "Male_actors"]

#Plot histograms for each independent variable
for i, col in enumerate(indep_var):
    sns.histplot(streaming_df_copy, x=col, kde=True, stat="density", ax=axes[i], bins=15)
    axes[i].set_title(f"Histogram of {col}")
    axes[i].set_xlabel(col)
    axes[i].set_ylabel("Frequency")

for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()


Release_year : The majority of movies in the dataset are very recent with an increase after the year 2000.
Duration: The duration of most movies is between 90 and 120 minutes, which is typical for films.
Is_Adult: Most movies are not classified as adult content but we can see that there is an important proportion of movies classified as adult content.
Female_count and male_count: We can see that the number of men is greater than women but this difference is lower than for CMU/IMDB movies.



In [ ]:
plt.figure(figsize=(14, 8))
for i, col in enumerate(numeric_columns_streaming, 1):
    plt.subplot(4, 2, i)
    sns.boxplot(data=streaming_df_copy, x=col)
    plt.title(f"Boxplot of {col}")
    
plt.suptitle("Figure 3: Boxplot of Variables")     
plt.tight_layout()
plt.show()

As we did in the previous part, we now have to focus on outliers and removing them for a more accurate modeling.

In [ ]:
# Define a function to remove outliers based on IQR
def remove_outliers(df, columns):
    for col in columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]
    return df


movie_df_no_outliers = remove_outliers(streaming_df_copy, numeric_columns_streaming)
plt.figure(figsize=(12, 8))
for i, col in enumerate(numeric_columns_streaming, 1):
    plt.subplot(4, 2, i)
    sns.boxplot(data=movie_df_no_outliers, x=col)
    plt.title(f"Boxplot of {col} (Without Outliers)")
plt.suptitle("Figure 4: Boxplot of Variables without Outliers")
plt.tight_layout()
plt.show()

In [ ]:
print(f"After removing the outliers and dropping the null values, we are left with {len(movie_df_no_outliers):,} movies.")

##### 1.1.2 Bivariate Analysis
a. <u>Correlation matrix</u>:

In [ ]:
#correlation matrix
correlation_matrix = movie_df_no_outliers[numeric_columns_streaming].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title("Figure 5: Correlation Matrix of Numeric Variables")
plt.show()

The number of votes and gender composition (especially the number of male actors) seem to influence movie duration. It suggests that popularity and cast composition could impact movie production characteristics.
Weak correlations between movie duration and other variables indicate that duration might not strongly determine popularity based on votes.
This correlation matrix suggests further analysis like regression that could help separate and clarify the effects of gender and duration on popularity.

b. <u>Scatter plots </u>:

In [ ]:
sns.pairplot(movie_df_no_outliers[numeric_columns_streaming])

### 5. <u>What are the social reasons behind the presence of female characters in movies? Is it due to sexualization or genuine equality of representation?</u>

The first part of our analysis uses plot summaries to examine whether female characters are portrayed as having agency (taking action and driving the story) or are reduced to sexualized or objectified roles. So far, we have used LLMs, such as ChatGPT Plus, to classify plot summaries based on these criteria (contains/does not contain sexualization of females), which provided useful initial results and validated the approach. However, due to time constraints during milestone 2, we were unable to use more advanced methods, such as zero-shot or few-shot learning with models like OpenAI's GPT-4, T5, or BERT variants, which we plan to explore further in milestone 3.
The second part of our analysis focuses on adult-rated films in the resulting merged dataframe, specifically examining the gender of the actors to understand patterns of representation.